# Database initialization

In [ ]:
# todo: create a script to build the logical model in the chosen NOSQL system

In [20]:
# Connection to Neo4j
from neo4j import GraphDatabase

URI = "bolt://127.0.0.1:7687"
USER = "neo4j"
PASSWORD = "neo4jproject123"

driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

with driver:
    driver.verify_connectivity()

ServiceUnavailable: Couldn't connect to 127.0.0.1:7687 (resolved to ('127.0.0.1:7687',)):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione)

In [18]:
query = """
MATCH (n) DETACH DELETE n
"""
run_query(query)
print("Database cleared.")


Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information

In [6]:
#Costrains
constraints = [
    "CREATE CONSTRAINT customer_id IF NOT EXISTS FOR (c:Customer) REQUIRE c.id IS UNIQUE",
    "CREATE CONSTRAINT terminal_id IF NOT EXISTS FOR (t:Terminal) REQUIRE t.id IS UNIQUE",
    "CREATE CONSTRAINT transaction_id IF NOT EXISTS FOR (tr:Transaction) REQUIRE tr.id IS UNIQUE"
]

for c in constraints:
    run_query(c)

print("Constraints created.")


Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information

In [3]:
#Nodes
def create_customer(customer):
    query = """
    MERGE (c:Customer {id: $id})
    SET c.location = $location,
        c.amountMean = $amountMean,
        c.amountStd = $amountStd,
        c.meanDailyTransactions = $meanDailyTransactions
    """
    run_query(query, customer)

def create_terminal(terminal):
    query = """
    MERGE (t:Terminal {id: $id})
    SET t.location = $location
    """
    run_query(query, terminal)

def create_transaction(transaction):
    query = """
    MERGE (tr:Transaction {id: $id})
    SET tr.dateTime = datetime($dateTime),
        tr.amount = $amount,
        tr.isFraud = $isFraud
    """
    run_query(query, transaction)


In [4]:
#Relationships
def link_customer_terminal(customer_id, terminal_id):
    query = """
    MATCH (c:Customer {id: $customer_id})
    MATCH (t:Terminal {id: $terminal_id})
    MERGE (c)-[:USE]->(t)
    """
    run_query(query, {"customer_id": customer_id, "terminal_id": terminal_id})

def link_customer_transaction(customer_id, transaction_id):
    query = """
    MATCH (c:Customer {id: $customer_id})
    MATCH (tr:Transaction {id: $transaction_id})
    MERGE (c)-[:PERFORM]->(tr)
    """
    run_query(query, {"customer_id": customer_id, "transaction_id": transaction_id})

def link_terminal_transaction(terminal_id, transaction_id):
    query = """
    MATCH (t:Terminal {id: $terminal_id})
    MATCH (tr:Transaction {id: $transaction_id})
    MERGE (t)-[:ON]->(tr)
    """
    run_query(query, {"terminal_id": terminal_id, "transaction_id": transaction_id})


In [ ]:
driver.close()
print("Connection closed.")
